In [493]:
import pandas as pd
import numpy as np
from bokeh.plotting import figure, output_file, show

In [494]:
train_df = pd.read_csv("train.csv")
train_df.dropna(inplace=True,axis=0,how='any')
#train_df.y.fillna(train_df.y.max(),inplace=True)
Y_train = train_df.y
X = train_df.x
m = Y_train.shape[0]
print("Y shape is : " , m)
train_df.head()

Y shape is :  699


,x,y
0,24.0,21.549452
1,50.0,47.464463
2,15.0,17.218656
3,38.0,36.586398
4,87.0,87.288984


In [495]:
#Plotting data
output_file("plot.html")
p = figure()
p.sizing_mode = 'scale_width'
p.circle(train_df.x,train_df.y,size=10,color='red',alpha=0.5)
show(p)

In [496]:
def computeCost(X,y,theta):
    size = y.shape[0]
    J = (1/(2*size)) * np.sum((np.matmul(X,theta) - y)**2)
    return J

In [497]:
def gradientDescent(X,y,theta,alpha,num_iter):
    print("initial theta shape is : ", theta)
    j = []
    #print("X contains null : " , np.isnan(Y).any())
    for i in range(num_iter):
        error = (np.dot(X,theta) - y)
        #print(np.isnan(error).any())
        theta = theta - ((alpha)*(np.dot(error,X)/y.shape[0]))
        j.append(computeCost(X,y,theta))
    print("theta is : ", theta)
        
    print("Cost is : " , computeCost(X,y,theta))
    return theta,j

In [498]:
X = np.transpose(X)

In [499]:
X = np.c_[np.ones(m),X]

In [500]:
print(X.shape)
print(np.sum(X))
print(np.isnan(Y_train).any())

(699, 2)
35659.0
False


In [512]:
theta = np.ones(X.shape[1])
iterations = 100000
alphas = [0.00005,0.00003,0.00001]
print(theta.shape)

(2,)


In [502]:
computeCost(X,Y_train,theta)

4.511264121372125

In [ ]:
J_dic = {}
theta_dic = {}
for i,alpha in enumerate(alphas):
    theta = np.ones(X.shape[1])
    theta,J = gradientDescent(X,Y_train,theta,alpha,iterations)
    J_dic[alpha] = J
    theta_dic[alpha] = theta

initial theta shape is :  [1. 1.]


In [504]:
#Plotting J against iterations data
output_file("Jplot.html")
p1 = figure()
p1.sizing_mode = 'scale_width'
for color, alpha in zip(['red','green','blue','yellow','purple'],alphas):
    p1.line(list(range(1,iterations+1)),J_dic[alpha],line_width=2,color=color,
            alpha=0.8,muted_color=color, muted_alpha=0.2,legend=str(alpha))
p1.xaxis.axis_label = "Iterations"
p1.yaxis.axis_label = "J"
p1.legend.location = "top_right"
p1.legend.click_policy="mute"
show(p1)

In [505]:
test_df = pd.read_csv("test.csv")
test_df.loc[test_df.y.isnull()]
X_test = np.transpose(test_df.x)
m_test = test_df.y.shape[0]
X_test = np.c_[np.ones(m_test),X_test]
predict = np.dot(X_test,theta_dic[0.00005])
correct = [(i,j) for i, j in zip(test_df.y.values,predict)]

In [506]:
def slope_intercept(x,y):
    slope = round(theta_dic[0.00005][1],2)
    b = np.mean(y) - np.dot(np.mean(x),slope)
    b = round(b,2)
    return slope,b

In [507]:
slope,b = slope_intercept(X, Y_train)

In [508]:
reg_line = [np.dot(slope,x) + b for x in train_df.x]

In [509]:
#Plotting data
output_file("plot2.html")
p2 = figure()
p2.sizing_mode = 'scale_width'
p2.circle(train_df.x,train_df.y,size=10,color='red',alpha=0.5)
p2.line(train_df.x,reg_line)
show(p2)